In [1]:
from __future__ import print_function
import pickle
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from oauth2client import client
from oauth2client import tools
from oauth2client.file import Storage
from apiclient.http import MediaFileUpload, MediaIoBaseDownload
import io
from apiclient import errors
from apiclient import http
import logging
from google.auth.transport.requests import Request
from getfilelistpy import getfilelist
import gspread
import pandas as pd
from oauth2client.service_account import ServiceAccountCredentials
import os
from pathlib import Path
from apiclient import discovery
from datetime import datetime
import re
import numpy as np
from pandas import DataFrame
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.backends.backend_pdf import PdfPages
from matplotlib import rcParams

os.chdir("/home/bruna_ramos/Downloads/data_science/annotation-routines/data/external/")

CLIENT_SECRET = "client_secrets.json"

SCOPES=[
    'https://www.googleapis.com/auth/drive']

creds = None
    # The file token.pickle stores the user's access and refresh tokens, and is
    # created automatically when the authorization flow completes for the first
    # time.
if os.path.exists('token.pickle'):
    with open('token.pickle', 'rb') as token:
        creds = pickle.load(token)
    # If there are no (valid) credentials available, let the user log in.
if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
        creds.refresh(Request())
    else:
        flow = InstalledAppFlow.from_client_secrets_file(CLIENT_SECRET, SCOPES)  # credentials.json download from drive API
        creds = flow.run_local_server()
        # Save the credentials for the next run
        with open('token.pickle', 'wb') as token:
            pickle.dump(creds, token)
service = build('drive', 'v3', credentials=creds)


#Com a pasta raíz informada identifica a pasta adicionada mais recentemente (batch atual) e traz as subpastas
def folders_to_current_batch(FOLDER_ID):
    
    """
    Extracts folders from current batch
    
    :param a: root folder
    :return: folders from current batch
    """

    folder_id=FOLDER_ID
    
# browse directories
    results_folder_1 = service.files().list( q="mimeType='application/vnd.google-apps.folder' and parents in '"+\
                                            folder_id+"' and trashed = false",fields=\
                                            "nextPageToken, files(id, name)",pageSize=400).execute()
#specify subfolder names    
    items_folder_1 = results_folder_1.get('files')
    for items in items_folder_1:
        if(items['name']=='sentenca'):
            folder_2=items['id']       
    results_folder_2 = service.files().list(q=\
                                            "mimeType='application/vnd.google-apps.folder' and parents in '"+\
                                            folder_2+"' and trashed = false",fields=\
                                            "nextPageToken, files(id, name)",pageSize=400).execute()
    items_folder_2 = results_folder_2.get('files', [])    
    for items in items_folder_2:
        if(items['name']=='semantix_anotacoes_sentencas_201911'):
            folder_3=items['id']
            

    #find current batch folder
    
    results_folder_3 = service.files().list(q="mimeType='application/vnd.google-apps.folder' and parents in '"+\
                                            folder_3+"' and trashed = false",fields=\
                                            "nextPageToken, files(id, name, createdTime)",\
                                            orderBy="createdTime desc").execute()
    items_folder_3 = results_folder_3.get('files', [])    
   

    #change folder id in case of previous batch reprocessing
    batch_atual=items_folder_3[0]['id']
    name_batch_atual=items_folder_3[0]['name']
 
    #batch_atual='1uUh2Pv8er4zw9bWcWfQtEV9WkYp_Womm'
    #name_batch_atual='batch_002'
    
    results_batch = service.files().list(q="mimeType='application/vnd.google-apps.folder' and parents in '"+\
                                         batch_atual+"' and trashed = false",fields=\
                                         "nextPageToken, files(id, name)",pageSize=400).execute()

    dic = results_batch.get('files', [])

    if not dic:
        print('No files found.')
    else:
        #print('Files:')
        total=[]
        for i in range(len(dic)):
            pastas=[]
            for key,value in dic[i].items():
                pastas.append((key,value,name_batch_atual))
            total.append(pastas)
    return total

def check_folder_to_annotator(folder_id,diretorio,pastas):
    
    """    
    check if all annotators have folders    
    :param a: folder id
    :param b: folder name
    :return: message of folders state
    """
   
    #authenticate access to google sheets
    os.chdir(diretorio)
    scope = ['https://spreadsheets.google.com/feeds']
    credentials = ServiceAccountCredentials.from_json_keyfile_name('py-sheets-e691f630640c.json', scope)
    gc = gspread.authorize(credentials)
    
    resource = {
        "oauth2": creds,
        "fields": "files(name,id)",
        "id":folder_id,
                }
    res = getfilelist.GetFileList(resource) 
    dic = res.get('fileList')

    if not dic:
        print('No files found.')
    else:
        for i in dic:

            for arquivo in i['files']:
                if 'annotators_list' in arquivo['name']:
                    lista_anotadores=arquivo

    ident=lista_anotadores['id']
    
   #create dataframe with file data 
    wks = gc.open_by_key(ident).sheet1
    data = wks.get_all_values()
    headers = data.pop(0)
    df = pd.DataFrame(data, columns=headers)
    
    #check if all annotators have folders 
    lista_anotadores=[]
    for anotador in df['anotadores']:
        lista_anotadores.append(anotador)
    
   
    lista_pastas=[]
    for pasta in pastas:
        lista_pastas.append((pasta[1][1]))

    lista_final = list(set(lista_anotadores) - set(lista_pastas ))

    if  not lista_final:
        print("\n Todos anotadores possuem pastas com seus nomes!")
    else:
        print("\n Nem todos anotadores possuem pastas, o diretório em falta é:",lista_final)



def find_files(folder_id,f_name):
    
    """
    Extracts folders from current batch
    
    :param a: folder id
    :param b: folder name
    :return: list of 'arquivos' files
    """
    folder_name = f_name
    resource = {
        "oauth2": creds,
        "fields": "files(name,id)",
        "id":folder_id,
                }
    res = getfilelist.GetFileList(resource)  

    dic = res.get('fileList')
    arquivos=[]
    if not dic:
        print('No files found.')
    else:
        for i in dic:
    #select only the 'arquivos' spreadsheet
            for arquivo in i['files']:
                if 'arquivos' in arquivo['name']:
                    arquivos.append(arquivo['name'])
    if len(arquivos)>1:
        print("\n Lista de arquivos",folder_name,arquivos)


def check_folders_and_files(folder_id,folder_name,folder_txt,folder_pdf,folder_xml):
         
    """    
    check if folders have expected files    
    :param a: folder id
    :param b: folder name
    :param c: txt folder name
    :param d: pdf folder name
    :param e: xml folder name
    :return: inconsistent files and folders
    """

    folder_name=folder_name

    results=service.files().list(q="mimeType='application/vnd.google-apps.folder' and parents in '"+folder_id+"' and trashed = false",fields="nextPageToken, files(id, name)",pageSize=400).execute()

    dic = results.get('files', [])
    list_folders=[]

    diretorio_ok=[]
    diretorio_nok=[]
    if not dic:
        print('No folders found.')
    else:
        for i in dic:
            list_folders.append(i['name'])
    if (folder_txt in list_folders) and (folder_xml in list_folders)\
    and (folder_pdf in list_folders):
         diretorio_ok.append((i['name'],i['id']))
    if (folder_txt  not in list_folders):
        print("\n A pasta 'arquivos_texto' não está disponível para:",folder_name)
        
    if ('arquivos_resultado' not in list_folders):
        print("\n A pasta 'arquivos_resultado' não está disponível para:",folder_name)
    if ('arquivos_originais' not in list_folders):
        print("\n A pasta 'arquivos_originais' não está criada para:",folder_name)
           


    #checks if there are unexpected shapes and classifications folders
    inconsistentes=[]
    for i in dic:
        f_name=i['name']
        f_id=i['id']
        
        resource = {"oauth2": creds,"fields": "files(name,id)", "id":f_id,"name":f_name}
        res = getfilelist.GetFileList(resource)

        dic = res.get('fileList')
        arquivos=[]
        arquivos_inconsistentes=[]
        
        if not dic:
            print('No files found.')
        else:
            for i in dic:
                if not i['files']:
                    print("\n Diretório","'",resource['name'],"'","vazio para",folder_name)
                else:
                    for x in i['files']:
                    
                        arquivos.append(x['name'])

                    if resource['name']==folder_txt:
                        for arquivo in arquivos:
                            if(arquivo[-4::]!='.txt')&(arquivo[-4::]!='.TXT') | ("(" in arquivo):
                                arquivos_inconsistentes.append((folder_name,resource['name'],arquivo))
                    if resource['name']==folder_pdf:
                        for arquivo in arquivos:
                            if(arquivo[-4::] !='.pdf' )&(arquivo[-4::] !='.PDF' )| ("(" in arquivo):
                                arquivos_inconsistentes.append((folder_name,resource['name'],arquivo))
                    if f_name==folder_xml:
                        for arquivo in arquivos:
                            if(arquivo[-4::] !='.xml')&(arquivo[-4::] !='.XML')| ("(" in arquivo):
                                arquivos_inconsistentes.append((resource['name'],arquivo,))
                inconsistentes.append(arquivos_inconsistentes)

    for i in inconsistentes:
        if i:
            print("\n Há inconsistencia de arquivos no seguinte diretorio:",i)
        else:
            pass

#Cria diretórios locais para disponibilizar o arquivo excel de cada anotador
def create_diretory(f_id,f_name,diretorio):
    
    """    
    create local directory    
    :param a: folder id
    :param b: folder name
    :param c: local directory address
    """
    
    dire=diretorio+f_name
    
    if not os.path.exists(dire):
        os.mkdir(dire)
        
def downloader(file_id,f_name,service):
    
    """    
    download files   
    :param a: file id
    :param b: file name
    :param c: autentication serivice name
    :return: progress download message
    """
    
    request = service.files().get_media(fileId=file_id)
    fh = io.FileIO(f_name, 'wb')
    downloader = MediaIoBaseDownload(fh, request)
    done = False
    while done is False:
        status, done = downloader.next_chunk()
        print('\n Download do arquivo',file_id,f_name,(status.progress() * 100))
        

def download_files_xlsx(Folder_id,f_name,service,diretorio,nome_arquivo):
    
    """    
    download files   
    :param a: folder id
    :param b: folder name
    :param c: local directory address
    :return: progress download message
    """
   
    name_folder=f_name
    folder_id=Folder_id
    os.chdir(diretorio+name_folder)
    
    results = service.files().list(q="mimeType='application/vnd.openxmlformats-officedocument.spreadsheetml.sheet' and parents in '"+folder_id+"' and trashed = false",fields="nextPageToken, files(id, name)").execute()
    items = results.get('files', [])
    for i in items:
        if nome_arquivo in i['name']:
            try:
                downloader(i['id'],i['name'],service)
            except IOError as e:
                print ("I/O error({0}): {1}".format(e.errno, e.strerror))


def concat_files(pastas,diretorio,arquivo):
    
    """    
    download files   
    :param a: list of folders
    :param b: local directory address
    :param c: downloaded file name
    :return: concatened files as dataframe
    """

    appended_data=[]
    for pasta in pastas:
        os.chdir(diretorio+pasta[1][1])
        if 'xlsx' in arquivo:
            df=pd.read_excel(arquivo)
            df.fillna('',inplace=True)
            appended_data.append(df)

    appended_data = pd.concat(appended_data)    
    appended_data.reset_index(inplace=True)    
    appended_data['batch_id']= int(pasta[0][2][-3::])    
    appended_data['xml_file']=False 
    regex="[0-9]{7}[-][0-9]{2}[\.][0-9]{4}[\.][0-9]{1}[\.][0-9]{2}[\.][0-9]{4}"
    appended_data['cnj']=appended_data.to.apply(lambda x:re.findall(regex,x)[0])    
    appended_data['batch_closing_date']=datetime.now()    
    appended_data = appended_data[['annotator','batch_closing_date','batch_id','cnj','date','id','prediction'\
                                  ,'to','xml_file']]
    appended_data.fillna("",inplace=True)
    appended_data['prediction'] =  appended_data['prediction'].str.strip()
    
    return appended_data

def gera_lista_arquivos_ja_trabalhados(line):
    
    """    
    creat a list of sentence worked files ids   
    :param a: dataframe line
    :return: list of worked files ids
    """
    
    lista_base=['sentenca_homologatoria','sentenca_execucao','sentenca','ata_julgamento']
    lista=[]
    if line.prediction in lista_base:
        lista=(line.id,line.annotator)
 
    return lista


def download_annotation_history(file_id,file_name,service,diretorio): 
    
    """    
    download annotation histoty file   
    :param a: file id
    :param b: file name
    :param c: service name
    :param d: local directory address
    :return: download progress
    """
    
    os.chdir(diretorio)
    try:
        downloader(file_id,file_name,service)
    except IOError as e:
        print ("I/O error({0}): {1}".format(e.errno, e.strerror))
        
        
        
def list_files_xml(Folder_id,pasta_mae,pasta_xml,lista_arquivos_trabalhados):
    
    """    
    select xml files  
    :param a: folder id
    :param b: root folder
    :param c: xml folder name
    :param d: worked files list
    :return: xml files
    """

    folder_id=Folder_id    
    folder_name=pasta_mae    
    results=service.files().list(q="mimeType='application/vnd.google-apps.folder' and parents in '"+folder_id+"' and trashed = false",fields="nextPageToken, files(id, name)",pageSize=400).execute()
    dic = results.get('files', [])
    
    for items in dic:
        if items['name']==pasta_xml:
            folder= items['id']

    results = service.files().list(q="mimeType='text/xml' and parents in '"+folder+"' and trashed = false",fields="nextPageToken, files(id, name)").execute()
    items = results.get('files', [])
    list_files_xml=[int(i['name'][0:6]) for i in items]
    l_trabalhado=[i[0] for i in lista_arquivos_trabalhados if i[1]== folder_name]
    arquivos_e_xml=list(set(list_files_xml) - set(l_trabalhado))
    
    if len(arquivos_e_xml)>1:
        print("\n Há inconsistencia entre a anotação da planilha 'arquivo.xlsx' e o(s) documento(s) xml:",arquivos_e_xml," da pasta:", folder_name)

    dicionario_xml = {}    
    dicionario_xml[folder_name]=list_files_xml

    return dicionario_xml

def load_drive(diretorio,file_id, nome_do_arquivo, mimetype):
    
    """    
    load files in google drive  
    :param a: local directory address
    :param b: file id
    :param c: file name
    :param d: file mimetype
    :return: upload status
    """

    data_modificacao = lambda f: f.stat().st_mtime
    directory = Path(diretorio)
    files = directory.glob(nome_do_arquivo)
    sorted_files = sorted(files, key=data_modificacao, reverse=True)
    arquivo =sorted_files[0]
    media_body = MediaFileUpload(
            arquivo, 
            mimetype=mimetype)
 # Send the request to the API.
    updated_file = service.files().update(
                                fileId=file_id,
                                media_body=media_body).execute()
    print('o seguinte arquivo foi atualizado:',updated_file)
    
def alertas(row):
    
    """    
    create a alerts list  
    :param a: dataframe line
    :param b: file id
    :param c: file name
    :param d: file mimetype
    :return: alerts list
    """
   
    alertas=[]
    #list of possible ratings
    lista_todos_arquivos=['acordao',
                        'agravo_instrumento',
                        'alvara_diversos',
                        'alvara_levantamento',
                        'ata_audiencia',
                        'ata_julgamento',
                        'contestacao',
                        'contrarrazoes',
                        'decisao_diversas',
                        'decisao_embargos',
                        'doc_diversos',
                        'embargos_declaratorios',
                        'embargos_execucao',
                        'erro_no_arquivo',
                        'guia_deposito',
                        'guia_diversos',
                        'peticao_inicial',
                        'peticao_intermediaria',
                        'peticao_juntada',
                        'recurso_ordinario',
                        'recurso_revista',
                        'sentenca',
                        'sentenca_homologatoria',
                        'sentenca_execucao']
    
    #list of classifications that must have xml file
    lista_xml=['sentenca_homologatoria','sentenca_execucao','sentenca','ata_julgamento']

    if row.classificacao_arquivo not in lista_xml and row.xml_file=='xml disponivel':
        alertas.append(("inconsistencia na pasta de resultados de: "+row.anotador+", batch "+(str(row.batch))\
                        +". Não é esperado arquivo xml para o documento do tipo: "\
                        +row.classificacao_arquivo+",id do documento:"+(str(row.id_documento))))
    if (row.data == ""):
        alertas.append((" Há data não preenchida para"+row.anotador+", batch "+(str(row.batch))\
                        +"documento"+(str(row.id_documento))))
    if row.classificacao_arquivo not in lista_todos_arquivos:
        alertas.append("Classificação inexistente de "+row.anotador+", batch "+(str(row.batch))+\
                       ", id_documento:"+(str(row.id_documento))+". A classificação '"+\
                       row.classificacao_arquivo+"' é inválida")
        
    return  alertas   

def write_txt(lista_alertas,diretorio):
    
    """ 
    create a txt file with inconsistencies in files and folders
    :param a: alerts list
    :param b: local directory address
    """
        
    with open(diretorio+'log_anotacoes.txt', 'w', encoding='utf-8') as f:
        for nome in lista_alertas:
            f.write(nome + '\n')    
   
    

def update_report(diretorio,file_id):
    
    """ 
    create a txt file with inconsistencies in files and folders
    :param a: local directory address
    :param b: file_id
    """
    
    # First retrieve the file from the API.
    file = service.files().get(fileId=file_id).execute()

    # File's new content.
    media_body = MediaFileUpload(
            'controle_anotacoes.xlsx', 
            mimetype='application/vnd.openxmlformats-officedocument.spreadsheetml.sheet')

    # Send the request to the API.
    updated_file = service.files().update(
                                fileId=file_id,

                                media_body=media_body).execute()
    print('\n O relatorio controle de anotações foi atualizado:',updated_file)



Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=887183825426-60s9l25rm89kp2ui22b1asrs58am4vgv.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8080%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&state=oQ59XLRjuhTDrnNFQqNUklalRp0NkA&access_type=offline


In [2]:
def main():
    
    #root folder id
    pastas = folders_to_current_batch('128GfOBbU_qW2OwHjJwadQfg9ZiHF0N9o')

    #root folder id
    check_folder_to_annotator('128GfOBbU_qW2OwHjJwadQfg9ZiHF0N9o',"/home/bruna_ramos/Downloads/",pastas)
    
    lista=[]
    for i in range(len(pastas)):
        lista.append(find_files(pastas[i][0][1],pastas[i][1][1]))
        
    lista_pastas_criadas=[]
    for i in range(len(pastas)):
        lista_pastas_criadas.append(check_folders_and_files(pastas[i][0][1],pastas[i][1][1],\
                                'arquivos_texto',\
                                'arquivos_originais',\
                                'arquivos_resultado'))
    
    # local directory address
    for pasta in pastas:
        create_diretory(pasta[0][1],pasta[1][1],"/home/bruna_ramos/Downloads/anotadores/")

    for pasta in pastas:
        download_files_xlsx(pasta[0][1],pasta[1][1],service,"/home/bruna_ramos/Downloads/anotadores/",
                            'arquivo')

    df=concat_files(pastas,"/home/bruna_ramos/Downloads/anotadores/","arquivos.xlsx")
        
    lista=df.apply(gera_lista_arquivos_ja_trabalhados,axis=1)
    lista_arquivos_trabalhados=[l for l in lista if l]


    download_annotation_history('1SS8TK_FFstAvAUpDSaUiuZbpKunQyIzU','annotation_history.csv',
                            service,"/home/bruna_ramos/Downloads/annotation_history/")



    os.chdir("/home/bruna_ramos/Downloads/annotation_history")

    df_annotation=pd.read_csv('annotation_history.csv')

    
    lista_xml=[]
    for pasta in pastas:
        lista_xml.append((list_files_xml(pasta[0][1],pasta[1][1],'arquivos_resultado',
                                         lista_arquivos_trabalhados)))
        
    # Defines in Dataframe which listed documents have xml files
    for index,row in df.iterrows(): 
        for i in range(len(lista_xml)):
            for key,value in lista_xml[i].items():
                if row.annotator==key and row.id in value:
                    df.at[index, 'xml_file'] = True

    df_trabalhado=df.loc[(df['prediction'] != '')]

    batch_atual=int(df_trabalhado.batch_id.unique())

    for index,row in df_annotation.iterrows():
        if row.batch_id == batch_atual:
             df_annotation.drop(index, inplace=True)
            
    df_annotation=df_annotation.append(df_trabalhado)
    
    df_report=df_annotation
    
   
    df_annotation= df_annotation[['annotator',
                                'batch_closing_date',
                                'batch_id','cnj',
                                'date','id',
                                'prediction','to','xml_file']]
        
    df_annotation.to_csv("/home/bruna_ramos/Downloads/annotation_history_upload/appended.csv")
    
    df_report=df_annotation
    
    df_report.fillna("",inplace=True)
    
    

    df_report.reset_index(drop=True, inplace=True)
    df_report['date_formated']=df_report.date.map(lambda x: str(x)[0:10] if x != "" else x)
   
    
    #load in google drive the anotation history file
    load_drive("/home/bruna_ramos/Downloads/annotation_history_upload",'1SS8TK_FFstAvAUpDSaUiuZbpKunQyIzU'\
               , '*.csv', 'text/csv')

    # define used columns
    df_report = df_report.rename(columns={'annotator':'anotador',
                                              'batch_closing_date': 'data_captura_dados',
                                             'batch_id':'batch',
                                             'cnj':'cnj',
                                             'id':'id_documento',
                                             'prediction':'classificacao_arquivo',
                                             'to':'diretorio_s3',
                                              'xml_file':'xml_file',
                                             'date_formated':'data'})

    df_report=df_report[['anotador','data_captura_dados','batch','cnj','id_documento','classificacao_arquivo',\
                  'diretorio_s3','xml_file','data']]

    df_report.loc[df_report["xml_file"]== False,'xml_file'] = 'sem xml'
    df_report.loc[df_report["xml_file"]== True,'xml_file'] = 'xml disponivel'
    df_report['classificacao_arquivo'] = df_report['classificacao_arquivo'].str.strip()
    

    rcParams.update({'figure.autolayout': True})
    
    # create a pdf with graphcs sequence
    with PdfPages("/home/bruna_ramos/Downloads/Charts.pdf") as export_pdf:

        donne_per_batch= df_report.loc[df_report['classificacao_arquivo']!=''].groupby(['batch'])['batch']\
.agg(['count']).reset_index()
        donne_per_batch = donne_per_batch.set_index(['batch'])
        all_files_per_batch=df_report.groupby('batch')['batch'].count()
        perc_done=(donne_per_batch['count']/all_files_per_batch)*100

        perc_done.plot(kind='barh', stacked=True,title="Volume de documentos já classificados - por batch\n")

        export_pdf.savefig()    
        plt.close()
    
        donne_per_annotator = df.loc[(df['prediction']!='')].groupby(['annotator'])\
        ['id'].agg(['count']).reset_index()
        do_per_annotator = df.loc[(df['id']!='')].groupby(['annotator'])\
        ['id'].agg(['count']).reset_index()
        donne_per_annotator= donne_per_annotator.merge(do_per_annotator, left_on='annotator', right_on='annotator')
        donne_per_annotator = donne_per_annotator.set_index(['annotator'])
        donne_per_annotator['feito']= donne_per_annotator['count_x']/donne_per_annotator['count_y']
        donne_per_annotator['meta'] = (donne_per_annotator['count_y'].sub(donne_per_annotator['count_x'], axis=0))/donne_per_annotator['count_y']

        donne_per_annotator.drop('count_x', inplace=True, axis=1)
        donne_per_annotator.drop('count_y', inplace=True, axis=1)
        donne_per_annotator.plot(kind='bar', stacked=True,\
                                 title="Documentos por anotador - batch atual\n",rot=90)
    
        export_pdf.savefig()    
        plt.close()
    
        donne_per_document = df.loc[(df['prediction']!='')].groupby(['prediction'])\
        ['id'].agg(['count']).sort_values(by ='count',ascending=False).reset_index()
        donne_per_document = donne_per_document.set_index(['prediction'])

        donne_per_document.plot(kind='bar', stacked=True,title="Classificação de documentos- batch atual\n",rot=90)

        export_pdf.savefig()    
        plt.close()
        
        all_donne_per_document = df_report.loc[(df_report['classificacao_arquivo']!='')].\
    groupby(['classificacao_arquivo'])['id_documento'].agg(['count']).sort_values(by ='count',\
                                                                              ascending=False).reset_index()
        all_donne_per_document = all_donne_per_document.set_index(['classificacao_arquivo'])

        all_donne_per_document.plot(kind='bar', stacked=True,title="CLassificação de documentos - todos os batches\n",rot=90)
        
        export_pdf.savefig()    
        plt.close()        
        
        sentenca_batchs=df_report.loc[df_report['classificacao_arquivo'].str.contains('sentenca')]\
        .groupby(['batch'])['batch'].agg(['count']).reset_index()
        sentenca_batchs = sentenca_batchs.set_index(['batch'])

        sentenca_batchs.plot(kind='barh', stacked=True,title\
                             ="Sentencas por batch\n",rot=0)
        
        export_pdf.savefig()    
        plt.close()
        
        
        all_files= df_report.loc[df_report['classificacao_arquivo']!=''].groupby(['batch'])['batch']\
.agg(['count']).reset_index()
        all_files = all_files.set_index(['batch'])
        only_sentence=df_report.loc[df_report['classificacao_arquivo'].str.contains('sentenca')].groupby(['batch'])['batch']\
.agg(['count']).reset_index()
        only_sentence = only_sentence.set_index(['batch'])
        donne_per_batch= all_files.merge(only_sentence, how="outer",on =['batch'])

        donne_per_batch = donne_per_batch.rename(columns={'count_x': 'total', 'count_y': 'sentenca'})
        donne_per_batch['%_sentenca']=donne_per_batch['sentenca']/donne_per_batch['total']*100
        donne_per_batch['%_nao_sentenca']= 100-donne_per_batch['%_sentenca']
        donne_per_batch.drop('total', inplace=True, axis=1)
        donne_per_batch.drop('sentenca', inplace=True, axis=1)

        donne_per_batch.plot(kind='barh', stacked=True,\
                             title="% Documentos classificados\n",rot=0)
        
        export_pdf.savefig()    
        plt.close()     
        
       
    
        doc_day= df_report.loc[(df_report['xml_file']=='xml disponivel') & \
                                 (df_report['classificacao_arquivo'].str.contains('sentenca'))].\
        groupby(['data'])['xml_file'].agg(['count']).sort_values(by='data').reset_index()
        
        ax=sns.catplot(x="data",y='count',kind='point',data=doc_day)
        plt.xlabel("data")
        plt.ylabel("contagem")
        plt.title("Sentenças com xml - todos batches")
        ax.fig.autofmt_xdate()
        export_pdf.savefig()    
        plt.close()
        
        docs_day_annotator= df_report.loc[(df_report['xml_file']=='xml disponivel') & \
                                 (df_report['classificacao_arquivo'].str.contains('sentenca'))].\
        groupby(['data','anotador'])['xml_file'].agg(['count']).sort_values(by='data').reset_index()
        
        datas=df_report['data'].unique()
        anotadores=df_report['anotador'].unique()
        index = pd.MultiIndex.from_product([datas, anotadores], names = ["data", "anotador"])

        stage_table=pd.DataFrame(index = index).sort_values(by='data').reset_index()
       
        docs_day_annotator= docs_day_annotator.merge(stage_table, how="outer",on =['data','anotador']).sort_values(by='data')
        docs_day_annotator['count'].fillna(0, inplace=True)

        g=sns.catplot(x="data",y='count',hue='anotador',kind='point',data=docs_day_annotator)
        plt.xlabel("data")
        plt.ylabel("contagem")
        plt.title("sentença com xml - todos batches")
        g._legend.remove()
        plt.legend(bbox_to_anchor=(1.0,1.0), title='anotador')
        g.fig.autofmt_xdate()
        export_pdf.savefig()    
        plt.close()
        
    #export charts   
    
    load_drive("/home/bruna_ramos/Downloads/",'1h064RuVBVGPCBK93cZ3q1BVPm8rZ1QRB',\
               'Charts.pdf', 'application/pdf')

    #tables to excel report
    
    analise = df_report.groupby(['batch','anotador','data','classificacao_arquivo', 'xml_file'])\
    ['xml_file'].agg(['count']).reset_index()
    
    per_day = df_report.groupby(['batch','data','anotador'])\
    ['data'].agg(['count']).reset_index()
    
    no_xml = df_report.loc[(df_report['xml_file']=='sem xml') & (df_report['classificacao_arquivo']\
                                                                 .str.contains('sentenca'))].\
    groupby(['batch','anotador','id_documento','classificacao_arquivo', 'xml_file'])\
    ['xml_file'].agg(['count']).reset_index()
    

    
    os.chdir("/home/bruna_ramos/Downloads/annotation_report/")

    writer = pd.ExcelWriter('controle_anotacoes.xlsx', engine='xlsxwriter')
# Write each dataframe to a different worksheet.
    analise.to_excel(writer, sheet_name='anotacoes')
    per_day.to_excel(writer, sheet_name='anotacoes_diarias')
    no_xml.to_excel(writer, sheet_name='anotacoes_sem_xml')

# Close the Pandas Excel writer and output the Excel file.
    writer.save()
    
    update_report("/home/bruna_ramos/Downloads/annotation_report/",'1PXYNB3kmHvKs_FB6lUgMW7HRiBDd0y1N')
    
# Gerar alertas de inconsistencias

    l_alertas=df_report.apply(alertas,axis=1)
    lista_alertas=[alerta[0] for alerta in l_alertas if alerta]
    
    write_txt(lista_alertas,"/home/bruna_ramos/Downloads/")  
    load_drive("/home/bruna_ramos/Downloads/",'1a90BmFj3bRqml5RagnB2dnjEHl_EEAaw',\
               'log_anotacoes.txt', 'text/plain')
    
    
main()


 Todos anotadores possuem pastas com seus nomes!

 Download do arquivo 1ZMSpDPCOG8sXrGdX8ELv8r19lwDP2hVF arquivos.xlsx 100.0

 Download do arquivo 1ZKY6lG8zfBuVIFvhKmeaD5KezCQYW7dk arquivos.xlsx 100.0

 Download do arquivo 1ZXwG4lbZkMX7iDIqGZng72ThEblFFaac arquivos.xlsx 100.0

 Download do arquivo 1tOXtEiKYBIl0bCtJd__RpgqffMVfbumK arquivos.xlsx 100.0

 Download do arquivo 1Z-hAOqbq-P-kS-bmlZOUmv5_2qkZL_gB arquivos.xlsx 100.0

 Download do arquivo 1Qauxeq_HPXcodYCp95dbu2lfsC-AsD8w arquivos.xlsx 100.0

 Download do arquivo 1xLwAG8AOiMkHb073BE6x_FU_jX95OW29 arquivos.xlsx 100.0

 Download do arquivo 1U9qiDPA4LhHIzu8zvxOW1ERNIvUPC5iq arquivos.xlsx 100.0


/home/bruna_ramos/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:361: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.




 Download do arquivo 1SS8TK_FFstAvAUpDSaUiuZbpKunQyIzU annotation_history.csv 100.0


/home/bruna_ramos/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:7138: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort,


o seguinte arquivo foi atualizado: {'kind': 'drive#file', 'id': '1SS8TK_FFstAvAUpDSaUiuZbpKunQyIzU', 'name': 'annotation_history.csv', 'mimeType': 'text/csv'}


/home/bruna_ramos/anaconda3/lib/python3.6/site-packages/pandas/core/ops/__init__.py:1115: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


o seguinte arquivo foi atualizado: {'kind': 'drive#file', 'id': '1h064RuVBVGPCBK93cZ3q1BVPm8rZ1QRB', 'name': 'controle_anotacoes.pdf', 'mimeType': 'application/pdf'}

 O relatorio controle de anotações foi atualizado: {'kind': 'drive#file', 'id': '1PXYNB3kmHvKs_FB6lUgMW7HRiBDd0y1N', 'name': 'controle_anotacoes.xlsx', 'mimeType': 'application/vnd.openxmlformats-officedocument.spreadsheetml.sheet'}


/home/bruna_ramos/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:511: FutureWarning: Series.data is deprecated and will be removed in a future version


o seguinte arquivo foi atualizado: {'kind': 'drive#file', 'id': '1a90BmFj3bRqml5RagnB2dnjEHl_EEAaw', 'name': 'log_anotacoes.txt', 'mimeType': 'text/plain'}
